In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw5.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 5: Putting it all together 
### Associated lectures: All material till lecture 13 

**Due date: [Monday, Mar 10, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative)**

## Table of contents
0. [Submission instructions](#si)
1. [Understanding the problem](#1)
2. [Data splitting](#2)
3. [EDA](#3)
4. [Feature engineering](#4)
5. [Preprocessing and transformations](#5) 
6. [Baseline model](#6)
7. [Linear models](#7)
8. [Different models](#8)
9. [Feature selection](#9)
10. [Hyperparameter optimization](#10)
11. [Interpretation and feature importances](#11) 
12. [Results on the test set](#12)
13. [Summary of the results](#13)
14. [Your takeaway from the course](#15)

<div class="alert alert-info">

## Submission instructions
<hr>
rubric={points:4}

**You may work with a partner on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).
- If you would like to use late tokens for the homework, all group members must have the necessary late tokens available. Please note that the late tokens will be counted for all members of the group.   


Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/master/docs/homework_instructions.md). 

1. Before submitting the assignment, run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Follow the [CPSC 330 homework instructions](https://ubc-cs.github.io/cpsc330-2024W2/docs/homework_instructions.html), which include information on how to do your assignment and how to submit your assignment.
4. Upload your solution on Gradescope. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
5. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope.


_Note: The assignments will get gradually more open-ended as we progress through the course. In many cases, there won't be a single correct solution. Sometimes you will have to make your own choices and your own decisions (for example, on what parameter values to use when they are not explicitly provided in the instructions). Use your own judgment in such cases and justify your choices, if necessary._

</div>

<!-- BEGIN QUESTION -->

## Imports

<div class="alert alert-warning">
    
Imports
    
</div>

_Points:_ 0

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier

<!-- END QUESTION -->

## Introduction <a name="in"></a>

In this homework you will be working on an open-ended mini-project, where you will put all the different things you have learned so far together to solve an interesting problem.

A few notes and tips when you work on this mini-project: 

#### Tips
1. This mini-project is open-ended, and while working on it, there might be some situations where you'll have to use your own judgment and make your own decisions (as you would be doing when you work as a data scientist). Make sure you explain your decisions whenever necessary. 
2. **Do not include everything you ever tried in your submission** -- it's fine just to have your final code. That said, your code should be reproducible and well-documented. For example, if you chose your hyperparameters based on some hyperparameter optimization experiment, you should leave in the code for that experiment so that someone else could re-run it and obtain the same hyperparameters, rather than mysteriously just setting the hyperparameters to some (carefully chosen) values in your code. 
3. If you realize that you are repeating a lot of code try to organize it in functions. Clear presentation of your code, experiments, and results is the key to be successful in this lab. You may use code from lecture notes or previous lab solutions with appropriate attributions. 

#### Assessment
We plan to grade fairly and leniently. We don't have some secret target score that you need to achieve to get a good grade. **You'll be assessed on demonstration of mastery of course topics, clear presentation, and the quality of your analysis and results.** For example, if you just have a bunch of code and no text or figures, that's not good. If you do a bunch of sane things and get a lower accuracy than your friend, don't sweat it.


#### A final note
Finally, this style of this "project" question is different from other assignments. It'll be up to you to decide when you're "done" -- in fact, this is one of the hardest parts of real projects. But please don't spend WAY too much time on this... perhaps "a few hours" (15-20 hours???) is a good guideline for this project . Of course if you're having fun you're welcome to spend as much time as you want! But, if so, try not to do it out of perfectionism or getting the best possible grade. Do it because you're learning and enjoying it. Students from the past cohorts have found such kind of labs useful and fun and I hope you enjoy it as well. 

<br><br>

<!-- BEGIN QUESTION -->

## 1. Pick your problem and explain the prediction problem <a name="1"></a>
<hr>
rubric={points:3}

In this mini project, you have the option to choose on which dataset you will be working on. The tasks you will need to carry on will be similar, independently of your choice.

### Option 1
You can choose to work on a classification problem of predicting whether a credit card client will default or not. 
For this problem, you will use [Default of Credit Card Clients Dataset](https://www.kaggle.com/uciml/default-of-credit-card-clients-dataset). In this data set, there are 30,000 examples and 24 features, and the goal is to estimate whether a person will default (fail to pay) their credit card bills; this column is labeled "default.payment.next.month" in the data. The rest of the columns can be used as features. You may take some ideas and compare your results with [the associated research paper](https://www.sciencedirect.com/science/article/pii/S0957417407006719), which is available through [the UBC library](https://www.library.ubc.ca/). 


### Option 2
You can choose to work on a regression problem using a [dataset](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data) of New York City Airbnb listings from 2019. As usual, you'll need to start by downloading the dataset, then you will try to predict `reviews_per_month`, as a proxy for the popularity of the listing. Airbnb could use this sort of model to predict how popular future listings might be before they are posted, perhaps to help guide hosts create more appealing listings. In reality they might instead use something like vacancy rate or average rating as their target, but we do not have that available here.

> Note there is an updated version of this dataset with more features available [here](http://insideairbnb.com/). The features were are using in `listings.csv.gz` for the New York city datasets. You will also see some other files like `reviews.csv.gz`. For your own interest you may want to explore the expanded dataset and try your analysis there. However, please submit your results on the dataset obtained from Kaggle.


**Your tasks:**

1. Spend some time understanding the options and pick the one you find more interesting (it may help spending some time looking at the documentation available on Kaggle for each dataset).
2. After making your choice, focus on understanding the problem and what each feature means, again using the documentation on the dataset page on Kaggle. Write a few sentences on your initial thoughts on the problem and the dataset. 
3. Download the dataset and read it as a pandas dataframe. 

<div class="alert alert-warning">
    
Solution_1
    
</div>

_Points:_ 3

For this assignment, I am picking Option 2 - predicting the amount of reviews_per_month for NYC Airbnb listings. Some initial thoughts: features like neighbourhood and price are very likely to effect our model. Some neighbourhoods are more desirable than others, and will have more people looking at listings. Price is also a factor - my prediction is that if it's too high or too low, less people will be interested.

In [3]:
df = pd.read_csv("AB_NYC_2019.csv")
print(df.shape)
df.head()

(48895, 16)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 2. Data splitting <a name="2"></a>
<hr>
rubric={points:2}

**Your tasks:**

1. Split the data into train (70%) and test (30%) portions with `random_state=123`.

> If your computer cannot handle training on 70% training data, make the test split bigger.  

<div class="alert alert-warning">
    
Solution_2
    
</div>

_Points:_ 2

In [4]:
train_df, test_df = train_test_split(df, train_size=0.7, test_size=0.3, random_state=123)
summ = train_df.describe(include='all')
summ

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
count,3.422600e+04,34216,3.422600e+04,34209,34226,34226,34226.000000,34226.000000,34226,34226.000000,34226.000000,34226.000000,27236,27236.000000,34226.000000,34226.000000
unique,NaN,33662,NaN,9201,5,218,NaN,NaN,3,NaN,NaN,NaN,1676,NaN,NaN,NaN
top,NaN,Home away from home,NaN,David,Manhattan,Williamsburg,NaN,NaN,Entire home/apt,NaN,NaN,NaN,2019-06-23,NaN,NaN,NaN
freq,NaN,14,NaN,292,15176,2728,NaN,NaN,17848,NaN,NaN,NaN,984,NaN,NaN,NaN
mean,1.893979e+07,NaN,6.726273e+07,NaN,NaN,NaN,40.729142,-73.952083,NaN,151.528399,7.094957,23.244814,NaN,1.369816,7.042453,112.526004
std,1.101332e+07,NaN,7.840511e+07,NaN,NaN,NaN,0.054531,0.046201,NaN,236.628392,21.548290,44.573323,NaN,1.700737,32.590803,131.420031
min,2.539000e+03,NaN,2.438000e+03,NaN,NaN,NaN,40.506410,-74.244420,NaN,0.000000,1.000000,0.000000,NaN,0.010000,1.000000,0.000000
25%,9.394482e+06,NaN,7.721897e+06,NaN,NaN,NaN,40.690193,-73.983030,NaN,69.000000,1.000000,1.000000,NaN,0.190000,1.000000,0.000000
50%,1.954546e+07,NaN,3.074526e+07,NaN,NaN,NaN,40.723240,-73.955550,NaN,106.000000,3.000000,5.000000,NaN,0.710000,1.000000,45.000000
75%,2.915085e+07,NaN,1.068375e+08,NaN,NaN,NaN,40.763287,-73.936270,NaN,175.000000,5.000000,23.000000,NaN,2.000000,2.000000,225.750000


In [5]:
missing_value_cols = ['name', 'host_name', 'last_review', 'reviews_per_month'] 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 3. EDA <a name="3"></a>
<hr>
rubric={points:10}

**Your tasks:**

1. Perform exploratory data analysis on the train set.
2. Include at least two summary statistics and two visualizations that you find useful, and accompany each one with a sentence explaining it.
3. Summarize your initial observations about the data. 
4. Pick appropriate metric/metrics for assessment. 

<div class="alert alert-warning">
    
Solution_3
    
</div>

_Points:_ 10

Since this is a regression problem, I will pick MSE and R-Squared as my assessment metrics.

The dataset shows that the `number_of_reviews` varies widely, with a mean of about 23 and a high standard deviation of 44, indicating that most listings have relatively few reviews, but some have a large number. The `price` column has a mean value of $151$, but a wide range, from $0$ to $10,000$, suggesting that the dataset includes both budget and luxury listings. The `availability_365` feature indicates that most listings are available for only part of the year (mean of 112 days), with a significant number available throughout the year (maximum of 365 days). The spread in `availability_365` suggests seasonal or varied hosting patterns.

In [6]:
# # Plot the neighbourhood_group distribution
# plt.figure(figsize=(8, 6))
# sns.countplot(x=X_train['neighbourhood_group'], data=X_train)
# plt.title('Distribution of Listings Across Neighborhood Groups')
# plt.xlabel('Neighborhood Group')
# plt.ylabel('Count of Listings')
# plt.xticks(rotation=45)
# plt.show()

This bar graph shows that the data mostly shows listings from the `Manhattan` and `Brooklyn` neighbourhoods. 

In [7]:
# # Plot the price distribution
# plt.figure(figsize=(8, 6))
# sns.histplot(X_train['price'], bins=50, kde=True)
# plt.title('Price Distribution of Listings')
# plt.xlabel('Price')
# plt.ylabel('Frequency')
# plt.show()

Most listing prices are $1000 or below. 

In [8]:
# # Scatter plot of Reviews per Month vs Price
# plt.figure(figsize=(8, 6))
# sns.scatterplot(x=X_train['price'], y=y_train)
# plt.title('Reviews per Month vs Price')
# plt.xlabel('Price')
# plt.ylabel('Reviews per Month')
# plt.show()

We see that there is some correlation with Price and Reviews per month, with most reviews being for listings with price < $2000.

In [9]:
# # List of numerical features
# # numerical_features = ['price', 'number_of_reviews', 'availability_365', 'longitude', 'latitude', 'minimum_nights', 'calculated_host_listings_count']

# # Set up the figure
# plt.figure(figsize=(15, 10))

# # Price vs Availability
# plt.subplot(2, 3, 1)
# sns.scatterplot(x=X_train['price'], y=X_train['availability_365'])
# plt.title('Price vs Availability')
# plt.xlabel('Price')
# plt.ylabel('Availability')

# # Price vs Longitude
# plt.subplot(2, 3, 2)
# sns.scatterplot(x=X_train['price'], y=X_train['longitude'])
# plt.title('Price vs Longitude')
# plt.xlabel('Price')
# plt.ylabel('Longitude')

# # Price vs Latitude
# plt.subplot(2, 3, 3)
# sns.scatterplot(x=X_train['price'], y=X_train['latitude'])
# plt.title('Price vs Latitude')
# plt.xlabel('Price')
# plt.ylabel('Latitude')

# # Price vs Minimum Nights
# plt.subplot(2, 3, 4)
# sns.scatterplot(x=X_train['price'], y=X_train['minimum_nights'])
# plt.title('Price vs Minimum Nights')
# plt.xlabel('Price')
# plt.ylabel('Minimum Nights')

# # Number of Reviews vs Calculated Host Listings Count
# plt.subplot(2, 3, 5)
# sns.scatterplot(x=X_train['number_of_reviews'], y=X_train['calculated_host_listings_count'])
# plt.title('Number of Reviews vs Host Listings Count')
# plt.xlabel('Number of Reviews')
# plt.ylabel('Host Listings Count')

# # Adjust layout
# plt.tight_layout()
# plt.show()


In [10]:
...

Ellipsis

In [11]:
...

Ellipsis

In [12]:
...

Ellipsis

In [13]:
...

Ellipsis

In [14]:
...

Ellipsis

In [15]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 4. Feature engineering <a name="4"></a>
<hr>
rubric={points:1}

**Your tasks:**

1. Carry out feature engineering. In other words, extract new features relevant for the problem and work with your new feature set in the following exercises. You may have to go back and forth between feature engineering and preprocessing. 

<div class="alert alert-warning">
    
Solution_4
    
</div>

_Points:_ 1

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 5. Preprocessing and transformations <a name="5"></a>
<hr>
rubric={points:10}

**Your tasks:**

1. Identify different feature types and the transformations you would apply on each feature type. 
2. Define a column transformer, if necessary. 

<div class="alert alert-warning">
    
Solution_5
    
</div>

_Points:_ 10

| Feature | Transformation | Explanation |
| --- | ----------- | ----- |
| **id** | drop | Unique identifier that does not provide predictive value. |
| **name** | drop | Free-text field with high cardinality, which is unlikely to contribute meaningfully to the prediction. |
| **host_id** | drop | Unique to each host, not useful for predicting reviews per month. |
| **host_name** | drop | High-cardinality categorical feature with little predictive value. |
| **neighbourhood_group** | one-hot encoding | Categorical variable representing a broad area |
| **neighbourhood** | one-hot encoding | More granular categorical feature for location; encoded to capture regional effects. |
| **latitude** | keep as is | Continuous numerical feature representing geographic position. |
| **longitude** | keep as is | Continuous numerical feature representing geographic position. |
| **room_type** | one-hot encoding | Categorical feature with a small number of categories |
| **price** | log transformation | Price has a highly skewed distribution; applying a log transformation reduces skewness and improves model performance. |
| **minimum_nights** | clip, log transformation | Outliers with extreme values need to be clipped; log transformation helps with skewness. |
| **number_of_reviews** | log transformation | Skewed distribution with some listings having an exceptionally high number of reviews. |
| **last_review** | convert to days since last review | Date feature converted into a numerical value representing recency of review. |
| **calculated_host_listings_count** | log transformation | Represents the number of listings a host manages; log transformation normalizes the distribution. |
| **availability_365** | keep as is | Continuous variable representing the number of available days in a year, which may directly impact review frequency. |


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.compose import make_column_transformer
import numpy as np
import pandas as pd

# Define transformation functions
log_transformer = FunctionTransformer(lambda x: np.log1p(x))  # log(1 + x) to handle zeros
clip_min_nights = FunctionTransformer(lambda x: np.clip(x, a_min=1, a_max=None))  # Clip minimum nights

# Define feature groups and transformations
drop_features = ["id", "name", "host_id", "host_name"]
one_hot_features = ["neighbourhood_group", "neighbourhood", "room_type"]
log_features = ["price", "number_of_reviews", "calculated_host_listings_count"]
clip_log_features = ["minimum_nights"]
date_feature = "last_review"
keep_as_is_features = ["latitude", "longitude", "availability_365"]

# Pipeline for one-hot encoding
one_hot_transformer = Pipeline([
    SimpleImputer(strategy="constant", fill_value="missing"),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline for log transformation
log_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("log_transform", log_transformer)
])

# Pipeline for minimum_nights (clipping + log transformation)
clip_log_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("clip_transform", clip_min_nights),
    ("log_transform", log_transformer)
])

date_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
])

# Define Column Transformer
preprocessor = make_column_transformer(
    (one_hot_transformer, one_hot_features),
    (log_pipeline, log_features),
    (clip_log_pipeline, clip_log_features),
    # (date_pipeline, [date_feature]),
    ("passthrough", keep_as_is_features),  # Keep certain numerical features as-is
    ("drop", drop_features)  # Drop irrelevant features
)

#Apply preprocessing pipeline
transformed_df = pd.DataFrame(preprocessor.fit_transform(X_train))

# Get transformed column labels manually
# For one-hot encoded columns
one_hot_columns = one_hot_features

# For log-transformed columns (no change in column names)
log_columns = log_features

# For clipped columns (no change in column names)
clip_columns = clip_log_features

# Passthrough columns remain unchanged
passthrough_columns = keep_as_is_features

# Combining all the columns together
transformed_columns = np.concatenate([
    one_hot_features, log_columns, clip_columns, passthrough_columns
])

# Create transformed DataFrame with correct column names
transformed_df = pd.DataFrame(imputed_data)
transformed_df

NameError: name 'X_train' is not defined

In [62]:
...

Ellipsis

In [ ]:
...

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 6. Baseline model <a name="6"></a>
<hr>
rubric={points:2}

**Your tasks:**
1. Try `scikit-learn`'s baseline model and report results.

<div class="alert alert-warning">
    
Solution_6
    
</div>

_Points:_ 2

In [50]:
X_train = train_df.drop(columns=['reviews_per_month'])  # Drop target column
y_train = train_df['reviews_per_month']  # Target column

X_test = test_df.drop(columns=['reviews_per_month'])  # Drop target column
y_test = test_df['reviews_per_month']  # Target column

# Summary statistics
summary_stats = X_train.describe()
summary_y = y_train.describe()
print(summary_stats[['number_of_reviews', 'price', 'availability_365']])
summary_y

       number_of_reviews         price  availability_365
count       34226.000000  34226.000000      34226.000000
mean           23.244814    151.528399        112.526004
min             0.000000      0.000000          0.000000
25%             1.000000     69.000000          0.000000
50%             5.000000    106.000000         45.000000
75%            23.000000    175.000000        225.750000
max           629.000000  10000.000000        365.000000
std            44.573323    236.628392        131.420031


count    27236.000000
mean         1.369816
std          1.700737
min          0.010000
25%          0.190000
50%          0.710000
75%          2.000000
max         58.500000
Name: reviews_per_month, dtype: float64

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 7. Linear models <a name="7"></a>
<hr>
rubric={points:10}

**Your tasks:**

1. Try a linear model as a first real attempt. 
2. Carry out hyperparameter tuning to explore different values for the complexity hyperparameter. 
3. Report cross-validation scores along with standard deviation. 
4. Summarize your results.

<div class="alert alert-warning">
    
Solution_7
    
</div>

_Points:_ 10

_Type your answer here, replacing this text._

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 8. Different models <a name="8"></a>
<hr>
rubric={points:12}

**Your tasks:**
1. Try at least 3 other models aside from a linear model. One of these models should be a tree-based ensemble model. 
2. Summarize your results in terms of overfitting/underfitting and fit and score times. Can you beat a linear model? 

<div class="alert alert-warning">
    
Solution_8
    
</div>

_Points:_ 12

_Type your answer here, replacing this text._

In [ ]:
...

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 9. Feature selection <a name="9"></a>
<hr>
rubric={points:2}

**Your tasks:**

Make some attempts to select relevant features. You may try `RFECV` or forward selection for this. Do the results improve with feature selection? Summarize your results. If you see improvements in the results, keep feature selection in your pipeline. If not, you may abandon it in the next exercises. 

<div class="alert alert-warning">
    
Solution_9
    
</div>

_Points:_ 2

_Type your answer here, replacing this text._

In [ ]:
...

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 10. Hyperparameter optimization <a name="10"></a>
<hr>
rubric={points:10}

**Your tasks:**

Make some attempts to optimize hyperparameters for the models you've tried and summarize your results. In at least one case you should be optimizing multiple hyperparameters for a single model. You may use `sklearn`'s methods for hyperparameter optimization or fancier Bayesian optimization methods. 
  - [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)   
  - [RandomizedSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
  - [scikit-optimize](https://github.com/scikit-optimize/scikit-optimize) 

<div class="alert alert-warning">
    
Solution_10
    
</div>

_Points:_ 10

_Type your answer here, replacing this text._

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 11. Interpretation and feature importances <a name="1"></a>
<hr>
rubric={points:10}

**Your tasks:**

1. Use the methods we saw in class (e.g., `shap`) (or any other methods of your choice) to examine the most important features of one of the non-linear models. 
2. Summarize your observations. 

<div class="alert alert-warning">
    
Solution_11
    
</div>

_Points:_ 10

_Type your answer here, replacing this text._

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 12. Results on the test set <a name="12"></a>
<hr>

rubric={points:10}

**Your tasks:**

1. Try your best performing model on the test data and report test scores. 
2. Do the test scores agree with the validation scores from before? To what extent do you trust your results? Do you think you've had issues with optimization bias? 
3. Take one or two test predictions and explain these individual predictions (e.g., with SHAP force plots).  

<div class="alert alert-warning">
    
Solution_12
    
</div>

_Points:_ 10

_Type your answer here, replacing this text._

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 13. Summary of results <a name="13"></a>
<hr>
rubric={points:12}

Imagine that you want to present the summary of these results to your boss and co-workers. 

**Your tasks:**

1. Create a table summarizing important results. 
2. Write concluding remarks.
3. Discuss other ideas that you did not try but could potentially improve the performance/interpretability . 
3. Report your final test score along with the metric you used at the top of this notebook in the [Submission instructions section](#si).

<div class="alert alert-warning">
    
Solution_13
    
</div>

_Points:_ 12

_Type your answer here, replacing this text._

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

<br><br>

<br><br>

<!-- BEGIN QUESTION -->

## 14. Your takeaway <a name="15"></a>
<hr>
rubric={points:2}

**Your tasks:**

What is your biggest takeaway from the supervised machine learning material we have learned so far? Please write thoughtful answers.  

<div class="alert alert-warning">
    
Solution_14
    
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
4. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
5. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

This was a tricky one but you did it! 

![](img/eva-well-done.png)